In [ ]:
import csv
import pandas as pd
import os
import json
from tqdm import tqdm
import numpy as np
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr, pointbiserialr
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, validation_curve, GridSearchCV, train_test_split, StratifiedKFold, KFold
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, make_scorer
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from category_encoders import TargetEncoder
import matplotlib.pyplot as plt
from itertools import product
from pprint import pprint
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import StandardScaler
from numpy import array  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re2 as re

In [8]:
all_df = pd.read_csv('all_df_350.csv', index_col=0)
feat_ad=pd.read_csv('adgraph/ad_features.csv',index_col=0)
label_ad=pd.read_csv('adgraph/ad_labelled.csv',index_col=0)
feat_web=pd.read_csv('webgraph/web_features.csv',index_col=0)
label_web=pd.read_csv('webgraph/web_labelled.csv',index_col=0)

df_labelled_ad = feat_ad.merge(label_ad[['visit_id', 'name', 'label']], on=['visit_id', 'name'])
df_labelled_ad = df_labelled_ad[df_labelled_ad['label'] != "Error"]

df_labelled_web = feat_web.merge(label_web[['visit_id', 'name', 'label']], on=['visit_id', 'name'])
df_labelled_web = df_labelled_web[df_labelled_web['label'] != "Error"]

df_word_embed = feat_ad.merge(label_ad[['visit_id', 'name', 'label', 'top_level_url']], on=['visit_id', 'name'])
df_word_embed = df_word_embed[df_word_embed['label'] != "Error"]
df_word_embed = df_word_embed[['visit_id', 'name', 'top_level_url', 'label']]

df_word_embed['top_level_url'] = df_word_embed['top_level_url'].str.replace("http://", "").str.replace("https://", "")
df_word_embed['top_level_url'] = df_word_embed['top_level_url'].str.replace("[", "%5B").str.replace("]", "%5D")
train, test = train_test_split(df_word_embed, test_size=0.2, random_state=42)

In [186]:
feat_cont = pd.DataFrame(df_labelled_ad.content_policy_type)
# feat_cont.reset_index(inplace=True, drop=True)
feat_cont['name'] = df_labelled_ad.content_policy_type
feat_cont.drop_duplicates(inplace = True, subset='content_policy_type', keep = "last")
feat_cont.set_index('content_policy_type',inplace=True)
feat_cont.to_dict()

ad_col = df_labelled_ad.columns
web_col = df_labelled_web.columns
col_list = []

for i in range(len(ad_col)):
    for j in range(len(web_col)):
        if ad_col[i] == web_col[j]:
            col_list.append(ad_col[i])
# col_list = col_list[2:-1]
df_outer_web = df_labelled_web.copy().drop(col_list, axis=1)
integ_df = pd.concat([df_labelled_ad.drop('label', axis=1), df_outer_web, df_labelled_ad['label']], axis=1)    ### AdGraph, WebGraph Features

_istarget=True
target_word_ft, target_word_ft_test = wordFeature(list(train['name']), list(test['name']), _istarget)
_istarget=False
source_word_ft, source_word_ft_test = wordFeature(list(train['top_level_url']), list(test['top_level_url']),_istarget)

fqdn_col = []
for i in range(0, 30):
    fqdn_col.append("fqdn_" + str(i))

source_word_ft.columns = fqdn_col
source_word_ft_test.columns = fqdn_col
    
req_col = []
for i in range(0, 200):
    req_col.append("req_" + str(i))
    
target_word_ft.columns = req_col
target_word_ft_test.columns = req_col

train_url230_sg = pd.concat([source_word_ft, target_word_ft], axis=1)
test_url230_sg = pd.concat([source_word_ft_test, target_word_ft_test], axis=1)

model load


100%|██████████| 166032/166032 [00:20<00:00, 8173.21it/s]


model load


100%|██████████| 166032/166032 [00:05<00:00, 29773.19it/s]


In [259]:
all_df_883 = pd.read_csv('all_df_883.csv', index_col=0)

new_features = all_df_883[all_df_883.columns[303:]].copy()
new_features.drop(['contentname', 'label'], axis=1, inplace=True)
new_features.fillna(0, inplace=True)

ngram_train, ngram_test = train_test_split(new_features, test_size=0.2, random_state=42)
ngram_train.reset_index(drop=True, inplace=True)
ngram_test.reset_index(drop=True, inplace=True)

integ_train, integ_test = train_test_split(integ_df, test_size=0.2, random_state=42)
integ_train.reset_index(drop=True, inplace=True)
integ_test.reset_index(drop=True, inplace=True)

train_df = pd.concat([integ_train[integ_train.columns[:-1]], train_url230_sg, ngram_train, integ_train['label']], axis=1)
test_df = pd.concat([integ_test[integ_test.columns[:-1]], test_url230_sg, ngram_test, integ_test['label']], axis=1)

sum_resolve = pd.read_csv('sum_resolve.csv', index_col=0)

In [9]:
train_resolve, test_resolve = train_test_split(sum_resolve, test_size=0.2, random_state=42)
train_resolve.reset_index(drop=True, inplace=True)
test_resolve.reset_index(drop=True, inplace=True)

In [2]:
train_df = pd.read_csv('all_df_883_train.csv', index_col=0)
test_df = pd.read_csv('all_df_883_test.csv', index_col=0)

In [11]:
train_df.loc[train_resolve.index, train_resolve.columns] = train_resolve
test_df.loc[test_resolve.index, test_resolve.columns] = test_resolve

In [14]:
# avg_indent = train_df['avg_ident'] ** -1
# for i in range(len(avg_indent)):
#     if avg_indent[i] == np.inf:
#         avg_indent[i] = 0
# train_df['avg_ident'] = avg_indent

# avg_indent = test_df['avg_ident'] ** -1
# for i in range(len(avg_indent)):
#     if avg_indent[i] == np.inf:
#         avg_indent[i] = 0
# test_df['avg_ident'] = avg_indent

# train_df.to_csv('all_df_883_train.csv')
# test_df.to_csv('all_df_883_test.csv')

In [15]:
Te = TargetEncoder()
Te.fit(train_df.content_policy_type, train_df.label)
train_df.content_policy_type = Te.transform(train_df.content_policy_type)
test_df.content_policy_type = Te.transform(test_df.content_policy_type)

In [ ]:
cnt = 0
filtered_feat = []
for i in train_df.columns[2:-1]:
    correlation, p_value = pointbiserialr(train_df.label, train_df[i])
    if ((p_value > 0.1)):
        print(i)
        print('Correlation coefficient:', correlation)
        print('p-value:', p_value)
        cnt += 1
        filtered_feat.append(i)
train_df.drop(filtered_feat, inplace=True, axis=1)
print('Removed : ', cnt)

In [153]:
X = train_df.drop(['visit_id', 'name', 'label'], axis=1)
y = train_df.label

In [258]:
temp_df = train_df[['avg_charperline',
'ast_breadth',
'ast_depth',
'brackettodot',
'avg_ident']]

temp_label = train_df.label

In [259]:
X_train, X_valid, y_train, y_valid = train_test_split(temp_df, temp_label, shuffle=True, test_size=0.2, random_state=42) # train / vaild
# Train a RandomForest model
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# Get feature importances
importances = clf.feature_importances_

# Convert the importances into a DataFrame
feature_importances = pd.DataFrame({"feature": X_train.columns.to_list(), "importance": importances})
print("feature_importances")
# Get permutation importances
result = permutation_importance(clf, X_valid, y_valid, n_repeats=5, random_state=42, n_jobs=-1)
print("result")
# Convert the importances into a DataFrame
perm_importances = pd.DataFrame({"feature": X_train.columns.to_list(), "importance": result.importances_mean})

# Print permutation importances
print(perm_importances)

feature_importances.columns = ['feature', 'RFI_importance']
perm_importances.columns = ['feature', 'PI_importance']
concat = feature_importances.copy()
concat = pd.concat([concat, perm_importances.PI_importance], axis=1)
concat['RFI_PI_MEAN'] = (concat.RFI_importance + concat.PI_importance) / 2
concat
concat.to_csv('RFI_PI_js.csv')

feature_importances
result
           feature  importance
0  avg_charperline    0.090857
1      ast_breadth    0.086631
2        ast_depth    0.052890
3     brackettodot    0.064933
4        avg_ident    0.075309


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42) # train / vaild
# Train a RandomForest model
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# Get feature importances
importances = clf.feature_importances_

# Convert the importances into a DataFrame
feature_importances = pd.DataFrame({"feature": X_train.columns.to_list(), "importance": importances})
print("feature_importances")
# Get permutation importances
result = permutation_importance(clf, X_valid, y_valid, n_repeats=5, random_state=42, n_jobs=-1)
print("result")
# Convert the importances into a DataFrame
perm_importances = pd.DataFrame({"feature": X_train.columns.to_list(), "importance": result.importances_mean})

# Print permutation importances
print(perm_importances)

feature_importances.columns = ['feature', 'RFI_importance']
perm_importances.columns = ['feature', 'PI_importance']
concat = feature_importances.copy()
concat = pd.concat([concat, perm_importances.PI_importance], axis=1)
concat['RFI_PI_MEAN'] = (concat.RFI_importance + concat.PI_importance) / 2
concat
concat.to_csv('RFI_PI_1006.csv')

In [162]:
graphFeatures=[  # 52
    "num_nodes",
    "num_edges",
    "nodes_div_by_edges",
    "edges_div_by_nodes",
    "in_degree",
    "out_degree",
    "in_out_degree",
    "average_degree_connectivity",
    "is_ancestor_script",
    "ascendant_has_ad_keyword",
    "descendant_of_eval_or_function",
    "ascendant_script_has_eval_or_function",
    "ascendant_script_has_fp_keyword",
    "ascendant_script_length",
    "ancestors",
    "descendants",
    "closeness_centrality",
    "eccentricity",
    "num_script_predecessors",
    "num_script_successors",
    "num_requests_received",
    "num_redirects_sent",
    "num_redirects_rec",
    "max_depth_redirect",
    "indirect_in_degree",
    "indirect_out_degree",
    "indirect_ancestors",
    "indirect_descendants",
    "indirect_closeness_centrality",
    "indirect_average_degree_connectivity",
    "indirect_eccentricity",
    "indirect_mean_in_weights",
    "indirect_min_in_weights",
    "indirect_max_in_weights",
    "indirect_mean_out_weights",
    "indirect_min_out_weights",
    "indirect_max_out_weights",
    "num_set_get_src",
    "num_set_mod_src",
    "num_set_url_src",
    "num_get_url_src",
    "num_set_get_dst",
    "num_set_mod_dst",
    "num_set_url_dst",
    "num_get_url_dst",
    "indirect_all_in_degree",
    "indirect_all_out_degree",
    "indirect_all_ancestors",
    "indirect_all_descendants",
    "indirect_all_closeness_centrality",
    "indirect_all_average_degree_connectivity",
    "indirect_all_eccentricity"
]

pi_is_zero = perm_importances.loc[(perm_importances.PI_importance <= 0)].feature.tolist()
X_wo_graph = X.drop(graphFeatures + pi_is_zero, axis = 1)

X_wo_graph = X_wo_graph[X_wo_graph.columns[:260]]
X_train, X_valid, y_train, y_valid = train_test_split(X_wo_graph, y, shuffle=True, test_size=0.2, random_state=42) # train / vaild

In [ ]:
# rf = RandomForestClassifier(random_state=42, n_estimators=100, n_jobs=-1)
# rf.fit(X_train, y_train)
# print("rf.fit(X_train, y_train)")
# result = permutation_importance(rf, X_train, y_train)
# perm_importance = result.importances_mean.argsort()

# rfi_pi_mean = concat['RFI_PI_MEAN'].copy()

min_features_to_select = 10
step = 1
selector = RFECV(clf, step=step, cv=5, min_features_to_select=min_features_to_select, scoring='accuracy', n_jobs=-1)
print("RFECV")

selector = selector.fit(X_wo_graph, y)
print("selector")

rfecv_support = selector.support_
rfecv_ranking = selector.ranking_
print("RFECV support:", rfecv_support)
print("RFECV ranking:", rfecv_ranking)
selected_perm_importance = perm_importances[rfecv_support]
print("Permutation importance for the selected features by RFECV:", selected_perm_importance)

mask = selector.get_support()
features = np.array(X_wo_graph.columns.to_list())
best_features = features[mask]

print("All features: ", X_wo_graph.columns.to_list())
print(features)

print("Selected best: ", best_features.shape[0])
print(features[mask])

In [ ]:
res_pearsonr = []
for i in range(len(X_wo_graph.columns)):
    for j in range(i, len(X_wo_graph.columns)):
        if pearsonr(X_wo_graph[X_wo_graph.columns[i]], X_wo_graph[X_wo_graph.columns[j]])[1] >= 0.05:
            print(f"{X_wo_graph.columns[i]}, {X_wo_graph.columns[j]} : {pearsonr(X_wo_graph[X_wo_graph.columns[i]], X_wo_graph[X_wo_graph.columns[j]])[1]}")
            if (X_wo_graph.columns[i].find('req_') != -1) | (X_wo_graph.columns[i].find('fqdn_') != -1):
                res_pearsonr.append(X_wo_graph.columns[i])
            else:
                res_pearsonr.append(X_wo_graph.columns[j])
                
res_spearman = []

for i in range(len(X_wo_graph.columns)):
    for j in range(i, len(X_wo_graph.columns)):
        if spearmanr(X_wo_graph[X_wo_graph.columns[i]], X_wo_graph[X_wo_graph.columns[j]]).pvalue >= 0.05:
            print(f"{X_wo_graph.columns[i]}, {X_wo_graph.columns[j]} : {spearmanr(X_wo_graph[X_wo_graph.columns[i]], X_wo_graph[X_wo_graph.columns[j]]).pvalue}")
            if (X_wo_graph.columns[i].find('req_') != -1) | (X_wo_graph.columns[i].find('fqdn_') != -1):
                res_spearman.append(X_wo_graph.columns[i])
            else:
                res_spearman.append(X_wo_graph.columns[j])

corr_list =pd.Series(res_pearsonr + res_spearman).unique()
corr_list

print(f"Final Selected Features : \n {[i for i in list(features[mask]) if i not in corr_list]} ")